# sfMix reading

We'll use this notebook to help read in sfMix experiments and manage the MountainSort spikes

Last change: 21.05.29   
Last opened: 21.05.29  

In [148]:
import os, sys, glob
import helper_fcns as hf
sys.path.insert(0, 'ExpoAnalysisTools/python/')
from ExpoAnalysisTools.python import read_sfMix, readBasicCharacterization
import numpy as np
import pandas as pd

In [149]:
import importlib as il

# LGN

#### For the LGN, we'll also need to take into account the classification (i.e. Magno/Parvo/Other). We can use Manu's excel sheet to do this.

In [ ]:
basePath = os.getcwd() + '/'
sheetName = 'manu-classification.xlsx';
# There are 3 sheets - m675, m680, m682
readIn = pd.read_excel(basePath + 'LGN/experiments/' + sheetName, sheet_name=['m675', 'm680', 'm682'])
def idToStr(typeId):
    if int(typeId) == 0:
        return 'parvo';
    elif int(typeId) == 1:
        return 'magno';
    else:
        return 'tbd'; # i.e. Manu says he couldn't classify definitively
    
readInFlat = pd.concat(readIn, keys=['m675', 'm680', 'm682']).reset_index()
# rename some things
readInFlat.rename(columns={'level_0': 'm#', 'level_1': 'expCount',
                          'celllabel': 'label', 'cellno': 'cellNum',
                          'expono': 'progNum'}, inplace=True)

# Are there NaN types?
nans = np.where(np.isnan(readInFlat['type']))[0]
readInFlat['label'][nans]

In [ ]:
# manually edit the NaN types...
readInFlat.at[12, 'type'] = 0 # parvo, per notes
readInFlat.at[18, 'type'] = 2 # TBD, per notes
readInFlat.at[19, 'type'] = 2 # TBD, per notes

In [ ]:
# Convert the index into a string with the type
readInFlat['typeStr'] = readInFlat['type'].apply(idToStr)
readInFlat['fullName'] = ['%s%s' % (x,y) for x,y in zip(readInFlat['m#'], readInFlat['label'])]

So, now we have a dataframe with the eccentricity and type for each recorded LGN cell. We can use this in the creation of the datalist to ensure that each cell has an associated eccentricity and type that will carry through all of the analyses.

### Convert the .xml to .npy!

In [ ]:
il.reload(read_sfMix);

In [ ]:
dl_name = 'dataList_210524.npy'
read_sfMix.read_sfMix_all('LGN/recordings/', dl_name=dl_name)

### Update the dataList
Remember that the previous function above creates/updates a dataList

In [150]:
# load the dataList
dl = hf.np_smart_load('LGN/structures/%s' % dl_name)
cellType = np.nan * np.zeros((len(dl['unitName'], )), dtype='O')

TypeError: list indices must be integers or slices, not str

In [ ]:
dl.keys()

In [ ]:
# Now, go through and find (in the readInFlat dataframe) the corresponding cell type, if there
for ii, nm in enumerate(dl['unitName']):
    split = nm.split('l');
    if len(split) == 1: # i.e. there was NOT an l in this name...
        
        mStr, unitNum = hf.parse_exp_name(nm)[0:2] # first two arguments
        
        validRow = np.where((readInFlat['m#'] == mStr) & (readInFlat['cellNum'] == unitNum))[0]
        if len(validRow)>0:
            cellType[ii] = readInFlat.at[validRow[0], 'typeStr']
        else:
            print('No match for %s' % nm)
    

dl['unitType'] = cellType
np.save('LGN/structures/%s' % dl_name, dl)

In [ ]:
for nm in ['parvo', 'magno', 'tbd']:
    ok = len(np.where(dl['unitType'] == nm)[0])
    print('%d %s cells' % (ok, nm))

# V1

In [151]:
il.reload(read_sfMix);

In [152]:
dl_name = 'dataList_210528.npy'
# read_sfMix.read_sfMix_all('V1/recordings/', dl_name=dl_name)

In [153]:
test_files = glob.glob('V1/structures/m*_sfm.npy')

unitNames = [];
expTypes = [];
unitArea = [];

for t in sorted(test_files):
    
    curr_file = hf.np_smart_load(t);
    
    unitNames.append(curr_file['unitLabel'])
    expName = curr_file['sfm']['exp']['filename'];
    expTypes.append(hf.parse_exp_name(expName)[-2]);
    unitArea.append('V1')
    
dataList = dict();
dataList['unitName'] = unitNames;
dataList['expType'] = expTypes
dataList['unitArea'] = unitArea;

print('%d cells' % len(unitNames))

np.save('V1/structures/%s' % dl_name, dataList)

11 cells
